In [193]:
import os
import spacy
import numpy as np
#
import pandas as pd
#
from spacy.matcher import Matcher

In [194]:
nlp = spacy.load('de_core_news_md')
pd.set_option('display.max_colwidth', None)
os.environ["SPACY_WARNING_IGNORE"] = "W008"

In [203]:
REVIEWS_CLEANED_FILE_PATH = 'data/labeled_data.csv'

df = pd.read_csv(REVIEWS_CLEANED_FILE_PATH, sep=';')
df.head()

,Unnamed: 0,caption,Essen (positiv),Essen (negativ),Service (positive),Service (negativ),Ambiente (positive),Ambiente (negative),Preis (positive),Preis (negativ),Wartezeit (positiv),Wartezeit (negativ),rating
0,0,"Die mit Abstand leckerste Pizza in Deutschland, und das selbst wenn man sie um halb 10 abends zum mitnehmen bestellt :) Einfach zum niederknien! …",1,0,1,0,0,0,0,0,0,0,5.0
1,4,Good place to dine in! Sadly no chicken pizzas available. Do keep in mind the waiting time when you visit✌🏻 …,1,0,0,1,0,0,0,0,0,1,4.0
2,5,This is the must try pizza in Frankfurt.,1,0,0,0,0,0,0,0,0,0,5.0
3,6,5/5 Pizza place in Frankfurt,1,0,0,0,0,0,0,0,0,0,5.0
4,7,Leckere Pizzen und coole Atmosphäre. 👍🏼 (,1,0,0,0,1,0,0,0,0,0,5.0


In [196]:
categories = ['essen', 'gaststättengewerbe', 'atmosphäre', 'preis', 'warten']

In [197]:
def spacy_most_similar(word, topn=10):
    ms = nlp.vocab.vectors.most_similar(
        nlp(word).vector.reshape(1,nlp(word).vector.shape[0]), n=topn)
    words = [nlp.vocab.strings[w] for w in ms[0][0]]
    distances = ms[2]
    return words, distances

In [198]:
spacy_most_similar(categories[0])

(['Aufessen',
  'weggefuttert',
  'drinken',
  'einzukochen',
  'koten',
  'inhaliere',
  'knabbere',
  'Bio-Essen',
  'schmatz',
  'Trinket'],
 array([[1.    , 0.8243, 0.8131, 0.7734, 0.7115, 0.7059, 0.7032, 0.6845,
         0.672 , 0.6681]], dtype=float32))

In [207]:
matcher = Matcher(nlp.vocab)
patterns = [[{'POS':'ADJ'}, {'POS':'NOUN'}]]
matcher.add("category", patterns)

##matches = matcher(nlp('ein gutes Fenster'))
#print(matches)


for rowIndex, wordText in enumerate(df['caption'][0:5]):
    print(wordText)
    doc = nlp(wordText)
    matches = matcher(doc)
    print(matches)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]
        span = doc[start:end]
        print(span.text)

Die mit Abstand leckerste Pizza in Deutschland, und das selbst wenn man sie um halb 10 abends zum mitnehmen bestellt :) Einfach zum niederknien! …
[(12545855305924804326, 3, 5)]
leckerste Pizza
Good place to dine in! Sadly no chicken pizzas available.  Do keep in mind the waiting time when you visit✌🏻 …
[]
This is the must try pizza in Frankfurt.
[]
5/5 Pizza place in Frankfurt
[]
Leckere Pizzen und coole Atmosphäre. 👍🏼  (
[(12545855305924804326, 0, 2), (12545855305924804326, 3, 5)]
Leckere Pizzen
coole Atmosphäre


In [200]:
similarity_min = 0.4

for rowIndex, wordText in enumerate(df['caption'][0:3]):
    word = nlp(wordText)
    tokens = [token.text for token in word]
    #tokens
    allowed_postags = ["NOUN"]
    for i, token in enumerate(word):
        if token.pos_ in allowed_postags:
            if token and token.vector_norm > 1:
                similarity_cat0 = nlp(categories[0]).similarity(token)
                if similarity_cat0 > similarity_min: # fits into
                    print(rowIndex, categories[0], token, similarity_cat0)

                similarity_cat1 = nlp(categories[1]).similarity(token)
                if similarity_cat1 > similarity_min: # fits into
                    print(rowIndex, categories[1], token, similarity_cat1)

                similarity_cat2 = nlp(categories[2]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[2], token, similarity_cat2)

                similarity_cat3 = nlp(categories[3]).similarity(token)
                if similarity_cat3 > similarity_min: # fits into
                    print(rowIndex, categories[3], token, similarity_cat3)

                similarity_cat4 = nlp(categories[4]).similarity(token)
                if similarity_cat2 > similarity_min: # fits into
                   print(rowIndex, categories[4], token, similarity_cat4)


0 essen Pizza 0.43865881948792484
0 essen mitnehmen 0.5011485718220593
2 essen pizza 0.43865881948792484


/var/folders/8f/b5rl_hw50bn0_k4xrt7mtm7w0000gn/T/ipykernel_43291/1235100666.py:15: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity_cat1 = nlp(categories[1]).similarity(token)


In [201]:
nlp("essen").similarity(nlp("pizza"))

0.4386588382204547

In [202]:
nlp('pizz').vector_norm

0